In [16]:

import numpy as np
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from itertools import combinations
from collections import Counter

In [ ]:
class OVAclasificador(BaseEstimator):

    def __init__(self, C=1.0, solver='lbfgs', max_iter=1000):
        self.C = C
        self.solver = solver
        self.max_iter = max_iter
        self.clf = None
        self.clases_ = None

    def Fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        base = LogisticRegression(C=self.C, solver=self.solver, max_iter=self.max_iter)
        self.clf = OneVsRestClassifier(base)
        self.clf.fit(X, y)
        self.clases_ = getattr(self.clf, 'classes_', None)
        return self

    def Predict(self, X):
        X = np.asarray(X)
        if self.clf is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        return self.clf.predict(X)

    def Predict_proba(self, X):
        X = np.asarray(X)
        if self.clf is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        if hasattr(self.clf, 'predict_proba'):
            return self.clf.predict_proba(X)
        if hasattr(self.clf, 'decision_function'):
            df = self.clf.decision_function(X)
            return 1 / (1 + np.exp(-df))
        raise AttributeError("El clasificador base no soporta predict_proba ni decision_function")

In [ ]:
class OVOclasificador(BaseEstimator):
    """One-vs-One wrapper delegando en sklearn OneVsOneClassifier."""

    def __init__(self, C=1.0, solver='lbfgs', max_iter=1000):
        self.C = C
        self.solver = solver
        self.max_iter = max_iter
        self.modelos = {}
        self.clases_ = None
        self.clf = None

    def Fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        base = LogisticRegression(C=self.C, solver=self.solver, max_iter=self.max_iter)
        self.clf = OneVsOneClassifier(base)
        self.clf.fit(X, y)
        self.clases_ = getattr(self.clf, 'classes_', None)
        estimators = getattr(self.clf, 'estimators_', None)
        if estimators is not None and self.clases_ is not None:
            for (a, b), est in zip(list(combinations(self.clases_, 2)), estimators):
                self.modelos[(a, b)] = est
        else:
            self.modelos = {}
        return self

    def Predict(self, X):
        X = np.asarray(X)
        if self.clf is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        return self.clf.predict(X)

In [ ]:
class SoftmaxRegression(BaseEstimator):
    """Regresión logística multinomial (softmax) implementada sin librerías externas.

    Variables clave y su significado/estado:
    - self.reg: coeficiente de regularización L2 (float). Si se pasa C, reg = 1/C.
    - self.lr: learning rate (float).
    - self.max_iter, self.tol: control de iteraciones y tolerancia para convergencia.
    - self.W: matriz de pesos de forma (d+1, K) donde d = n_features, K = n_clases.
        - fila 0 corresponde al bias/intercept para cada clase.
        - se inicializa con ceros y se actualiza por descenso por gradiente.
    - self.classes_: array de etiquetas únicas (shape (K,)).

    Durante Fit:
    - Xb: X con columna de 1's añadida (shape (n, d+1)).
    - Y: one-hot (n, K) con 1 en la columna de la clase verdadera.
    - scores = Xb.dot(W): (n, K) logits antes de softmax.
    - probs: softmax(scores) -> (n, K) probabilidades por clase.
    - grad: gradiente de la pérdida (d+1, K) incluyendo término de regularización
    - reg_term: término que penaliza W (no el bias), de forma (d+1, K).
    - diff: norma de cambio de W entre iteraciones (usada para el criterio de parada)
    """

    def __init__(self, C=None, lr=0.1, max_iter=1000, tol=1e-5, reg=None, verbose=False, **kwargs):
        if reg is None:
            if C is None:
                self.reg = 1e-3
            else:
                self.reg = 1.0 / C
        else:
            self.reg = reg
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol
        self.verbose = verbose
        self.W = None
        self.classes_ = None

    def _one_hot(self, y_idx, K):
        n = y_idx.shape[0]
        Y = np.zeros((n, K))
        Y[np.arange(n), y_idx] = 1
        return Y

    def Fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.classes_ = np.unique(y)
        K = len(self.classes_)
        class_to_idx = {c: i for i, c in enumerate(self.classes_)}
        y_idx = np.vectorize(class_to_idx.get)(y)

        n, d = X.shape
        Xb = np.hstack([np.ones((n, 1)), X])

        self.W = np.zeros((d + 1, K))

        Y = self._one_hot(y_idx, K)

        for it in range(self.max_iter):
            scores = Xb.dot(self.W)
            scores -= scores.max(axis=1, keepdims=True)
            exp_scores = np.exp(scores)
            probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)

            grad = - (Xb.T.dot(Y - probs)) / n
            reg_term = self.reg * np.vstack([np.zeros((1, K)), self.W[1:, :]])
            grad += reg_term

            W_old = self.W.copy()
            self.W -= self.lr * grad

            diff = np.linalg.norm(self.W - W_old)
            if self.verbose and (it % 100 == 0 or it == self.max_iter - 1):
                loss = -np.mean(np.sum(Y * np.log(probs + 1e-15), axis=1)) + 0.5 * self.reg * np.sum(self.W[1:, :] ** 2)
                print(f"it={it} loss={loss:.6f} ||dW||={diff:.6e}")
            if diff < self.tol:
                break
        return self

    def Predict(self, X):
        if self.W is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        X = np.asarray(X)
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])
        scores = Xb.dot(self.W)
        scores -= scores.max(axis=1, keepdims=True)
        exp_scores = np.exp(scores)
        probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)
        idx = np.argmax(probs, axis=1)
        return self.classes_[idx]

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

data = load_iris()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

ova = OVAclasificador()
ovo = OVOclasificador()
try:
    sm = SoftmaxRegression(lr=0.5, max_iter=2000, tol=1e-7, verbose=False, reg=1e-3)
except TypeError:
    sm = SoftmaxRegression(C=1.0, solver='lbfgs', max_iter=2000)

models = [('OvA', ova), ('OvO', ovo), ('Softmax', sm)]

results = {}

for name, model in models:
    t0 = time.time()
    model.Fit(X_train_s, y_train)
    y_pred = model.Predict(X_test_s)
    t1 = time.time()
    acc = accuracy_score(y_test, y_pred)
    results[name] = {'accuracy': acc, 'time_s': t1 - t0, 'y_pred': y_pred}
    print(f"{name}: accuracy={acc:.4f} time={t1-t0:.3f}s")
    print(classification_report(y_test, y_pred, target_names=data.target_names))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("-" * 60)

print("Resumen:")
for name, info in results.items():
    print(f"{name}: acc={info['accuracy']:.4f} time={info['time_s']:.3f}s")

OvA: accuracy=0.8444 time=0.025s
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.79      0.73      0.76        15
   virginica       0.75      0.80      0.77        15

    accuracy                           0.84        45
   macro avg       0.85      0.84      0.84        45
weighted avg       0.85      0.84      0.84        45

Confusion matrix:
 [[15  0  0]
 [ 0 11  4]
 [ 0  3 12]]
------------------------------------------------------------
OvO: accuracy=0.9111 time=0.010s
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.82      0.93      0.88        15
   virginica       0.92      0.80      0.86        15

    accuracy                           0.91        45
   macro avg       0.92      0.91      0.91        45
weighted avg       0.92      0.91      0.91        45

Confusion matrix:
 [[15  0  0]
 [ 0 14  1]
 [ 0  3 12]]